# Demo requirements

This notebook creates a dummy model to use for inference and registers the dataset to use for inference.

In [ ]:
# Variables used in script
model_name = "simple-model"
dataset_name = "simple-batch"

## Create dummy model and register it in AzureML

In [ ]:
from SimpleModel import SimpleModel
# Create an instance of the simple model that counts the lenght
# of the specific feature
dummyModel = SimpleModel('word')

In [ ]:
import pandas as pd
df = pd.read_csv("sampledata.csv")
df.head()

In [ ]:
# And test that the output 
df['outputs']=dummyModel.predict(df)
df.head()

In [ ]:
# Create a folder to store the memory dump of the dummy model
import os

model_path = 'model'

if not os.path.exists(model_path):
    os.makedirs(model_path) 

In [ ]:
import joblib

# Store  model as joblib dump
joblib.dump(value=dummyModel, filename=os.path.join('model','model.joblib'))


In [ ]:
from azureml.core import Workspace

# Connect to workspace
ws = Workspace.from_config()

In [ ]:
from azureml.core import Model

model = Model.register(ws,
                model_name=model_name,
                model_path=model_path
                )

## Register dummy dataset to evaluate

In [ ]:
datastore = ws.get_default_datastore()

# Upload csv file in the dataset_name folder of the default data store
datastore.upload_files(['sampledata.csv'], target_path=f'{dataset_name}/')

In [ ]:
from azureml.core import Dataset

# Create an unregistered dataset reference to the file we uploaded
csv_files = [(datastore, f"{dataset_name}/sampledata.csv")]
dataset = Dataset.Tabular.from_delimited_files(path=csv_files)

In [ ]:
# Register the dataset to get reference during the inference pipeline
# You do not need to register the dataset as you can create the reference 
# as we did above during the batch inference execution

dataset.register(workspace=ws,
                 name=dataset_name,
                 description=f"Simple data simple batch inference pipeline")